캐글 데이터 경로 확인

In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./data/sample_submission.csv
./data/test/ngr/00829.png
./data/test/ngr/00830.png
./data/test/ngr/00831.png
./data/test/ngr/00832.png
./data/test/ngr/00833.png
./data/test/ngr/00834.png
./data/test/ngr/00835.png
./data/test/ngr/00836.png
./data/test/ngr/00837.png
./data/test/ngr/00838.png
./data/test/ngr/00839.png
./data/test/ngr/00840.png
./data/test/ngr/00841.png
./data/test/ngr/00842.png
./data/test/ngr/00843.png
./data/test/ngr/00844.png
./data/test/ngr/00845.png
./data/test/ngr/00846.png
./data/test/ngr/00847.png
./data/test/ngr/00848.png
./data/test/ngr/00849.png
./data/test/ngr/00850.png
./data/test/ngr/00851.png
./data/test/ngr/00852.png
./data/test/ngr/00853.png
./data/test/ngr/00854.png
./data/test/ngr/00855.png
./data/test/ngr/00856.png
./data/test/ngr/00857.png
./data/test/ngr/00858.png
./data/test/ngr/00859.png
./data/test/ngr/00860.png
./data/test/ngr/00861.png
./data/test/ngr/00862.png
./data/test/ngr/00863.png
./data/test/ngr/00864.png
./data/test/ngr/00865.png
./data/te

### 폴더 경로 설정

In [41]:
workspace_path = './data'  # 본인의 파일 경로 반영

### 필요한 패키지 로드

In [42]:
!pip install --upgrade pip
!pip install -U albumentations
# !pip install albumentations==0.4.6
# !pip install yacs

In [43]:
import os
import torch
import torchvision
import torch.nn.functional as F
import yaml
import numpy as np
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
import torch.backends.cudnn as cudnn
import time
from tqdm import tqdm

### 재구현 세팅

In [44]:
def init_seeds(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Speed-reproducibility tradeoff https://pytorch.org/docs/stable/notes/randomness.html
    if seed == 0:  # slower, more reproducible
        cudnn.deterministic = True
        cudnn.benchmark = False
    else:  # faster, less reproducible
        cudnn.deterministic = False
        cudnn.benchmark = True

In [45]:
init_seeds(0)

### 데이터 로드

In [46]:
rgb_path = os.path.join(workspace_path, 'train/rgb/')
ngr_path = os.path.join(workspace_path, 'train/ngr/')
label_path = os.path.join(workspace_path, 'train/label/')

In [47]:
rgb_images = os.listdir(rgb_path)
rgb_images = [os.path.join(rgb_path,x) for x in rgb_images]
ngr_images = os.listdir(ngr_path)
ngr_images = [os.path.join(ngr_path, x) for x in ngr_images]
label_images = os.listdir(label_path)
label_images = [os.path.join(label_path, x) for x in label_images]

### 데이터셋 클래스 정의

In [48]:
class CloudDataset(torch.utils.data.Dataset):
    def __init__(self, image_path, label_path, patch_size = 400, patch_stride = 100, is_train = True, cache_dir = './cache', transforms = None):
        self.image_path = image_path
        self.label_path = label_path
        self.patch_size = patch_size
        self.patch_stride = patch_stride
        self.is_train = is_train
        self.transforms = transforms
        
        self.patch_images = []
        self.patch_labels = []
        
        
        cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        if is_train:
            for img_path in self.image_path:
                img = cv2.imread(img_path)
                img_count = 0
                for x in range(0, img.shape[0]-self.patch_size+1, self.patch_stride):
                    for y in range(0, img.shape[1]-self.patch_size+1, self.patch_stride):
                        patch_image = img[x:x+patch_size, y:y+patch_size, :].copy()
                        patch_path = f'rgb_{os.path.splitext(os.path.basename(img_path))[0]}_{img_count}.png'
                        if not os.path.isfile(os.path.join(cache_dir, patch_path)):
                            cv2.imwrite(os.path.join(cache_dir, patch_path), patch_image)
                        self.patch_images.append(os.path.join(cache_dir, patch_path))
                        img_count += 1

            for label_path in self.label_path:
                img = cv2.imread(label_path)
                img_count = 0
                for x in range(0, img.shape[0]-self.patch_size+1, self.patch_stride):
                    for y in range(0, img.shape[1]-self.patch_size+1, self.patch_stride):
                        patch_image = img[x:x+patch_size, y:y+patch_size, :].copy()
                        patch_path = f'label_{os.path.splitext(os.path.basename(label_path))[0]}_{img_count}.png'
                        if not os.path.isfile(os.path.join(cache_dir, patch_path)):
                            cv2.imwrite(os.path.join(cache_dir, patch_path), patch_image)
                        self.patch_labels.append(os.path.join(cache_dir, patch_path))
                        img_count += 1
        else:
            self.patch_images = self.image_path
            self.patch_labels = self.label_path
    def __len__(self):
        return len(self.patch_images)
        
    def __getitem__(self, idx):
        img = cv2.imread(self.patch_images[idx])
        
        if self.is_train:
            label = cv2.imread(self.patch_labels[idx])
            # numpy arrays to tensors
            h, w = label.shape[:2]
        
            target = np.zeros((h, w), dtype=np.uint8)
            pos = np.where(np.all(label == [0, 0, 255], axis=-1))  # thick cloud
            target[pos] = 1
            pos = np.where(np.all(label == [0, 255, 0], axis=-1))  # thin cloud
            target[pos] = 2
            pos = np.where(np.all(label == [0, 255, 255], axis=-1))  # cloud shadow
            target[pos] = 3
        else:
            target = img
        if self.transforms is not None:
            img, target = self.transforms(img, target)
            
        if self.is_train:
            return img, target
        else:
            return img, self.patch_images[idx]

### 파라미터 세팅

In [49]:
batch_size = 8
epochs = 20
device = 'cuda' if torch.cuda.is_available() else 'cpu'
patch_size = 400
patch_stride = 100
num_workers = 0

num_classes = 4
class_names = ['thick cloud', 'thin cloud', 'cloud shadow']

train_data_rate = 0.8

model_name = 'deeplabv3'

loss_func = 'dice'

### 데이터증대

In [50]:
class ImageAug:
    def __init__(self):
        self.aug = A.Compose([A.HorizontalFlip(p=0.5),
                             A.VerticalFlip(p=0.5),
                             A.ShiftScaleRotate(p=0.5),
                             A.RandomBrightnessContrast(p=0.3),
                             A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.3),
                             A.RandomGamma(p=0.2),
                             A.CLAHE(p=0.3),
                             A.GaussNoise(p=0.3),
                             A.Downscale(p=0.3),
                            #  A.Sharpen(p=0.3),
                             A.Normalize(),
                             ToTensorV2()])

    def __call__(self, img, label):
        transformed = self.aug(image=img, mask=label)
        return transformed['image'], transformed['mask']

class DefaultAug:
    def __init__(self):
        self.aug = A.Compose([A.Normalize(),
                             ToTensorV2()])

    def __call__(self, img, label):
        transformed = self.aug(image=img, mask=label)
        return transformed['image'], transformed['mask']

In [51]:
train_transforms = ImageAug()
val_transforms = DefaultAug()

### 데이터셋 정의

In [52]:
output_path = './kaggle/working'
#train dataset
# train_dataset = CloudDataset(rgb_images[:int(len(rgb_images)*train_data_rate)], label_images[:int(len(label_images)*train_data_rate)],
#                             transforms=train_transforms, cache_dir=os.path.join(output_path, 'cache'))
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
#                                                num_workers=num_workers, pin_memory=True, drop_last=True)

# #valid dataset
# val_dataset = CloudDataset(rgb_images[int(len(rgb_images)*train_data_rate):], label_images[int(len(label_images)*train_data_rate):],
#                             transforms=val_transforms, cache_dir=os.path.join(output_path, 'cache'))
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
#                                                num_workers=num_workers, pin_memory=True, drop_last=True)

### 모델 정의

In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [54]:
import torch.nn as nn

class DoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConvBlock, self).__init__()
        self.block = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                                   nn.ReLU(inplace=True),
                                   nn.BatchNorm2d(out_channels),
                                   nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                                   nn.ReLU(inplace=True),
                                   nn.BatchNorm2d(out_channels))

    def forward(self, x):
        x = self.block(x)
        return x


class DilatedConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dilation, padding):
        super(DilatedConvBlock, self).__init__()
        self.block = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=padding, dilation=dilation),
                                   nn.ReLU(inplace=True),
                                   nn.BatchNorm2d(out_channels))

    def forward(self, x):
        x = self.block(x)
        return x




class ConcatDoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConcatDoubleConvBlock, self).__init__()
        self.block = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                                   nn.ReLU(inplace=True),
                                   nn.BatchNorm2d(out_channels),
                                   nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                                   nn.ReLU(inplace=True),
                                   nn.BatchNorm2d(out_channels))

    def forward(self, x, skip):
        x = torch.cat((skip, x), dim=1)
        x = self.block(x)
        return x



class MyDilatedConvUNet(nn.Module):
    def __init__(self, filters=44, depth=3, bottleneck_depth=6):
        super(MyDilatedConvUNet, self).__init__()
        self.depth = depth
        self.encoder_path = nn.ModuleList()
        src_in_channels = 3     # Geo-TIFF has four channels (R, G, B, and NIR)
        for d in range(depth):
            in_channels = src_in_channels if d == 0 else filters * 2 ** (d-1)
            self.encoder_path.append(
                DoubleConvBlock(in_channels, filters * 2 ** d))
        self.maxpool = nn.MaxPool2d(2, 2, padding=0)
        self.bottleneck_path = nn.ModuleList()
        for d in range(bottleneck_depth):
            in_channels = filters * 2 ** (depth - 1) if d == 0 else filters * 2 ** depth
            self.bottleneck_path.append(DilatedConvBlock(in_channels, filters * 2 ** depth, 2 ** d, 2 ** d))
        self.decoder_path = nn.ModuleList()
        for d in range(depth):
            in_channels = filters * 2 ** (depth - d)
            self.decoder_path.append(ConcatDoubleConvBlock(in_channels, filters * 2 ** (depth - d - 1)))
        self.up_path = nn.ModuleList()
        for d in range(depth):
            in_channels = filters * 2 ** (depth - d)
            self.up_path.append(nn.ConvTranspose2d(in_channels, filters * 2 ** (depth - d - 1),
                                                        kernel_size=4, stride=2, padding=1))
        out_channels = 4     # output channels (num_classes + 1(background))
        self.last_conv = nn.Conv2d(filters, out_channels, kernel_size=1)

    def forward(self, x):
        skip = []
        for block in self.encoder_path:
            x = block(x)
            skip.append(x)
            x = self.maxpool(x)
        dilated = []
        for block in self.bottleneck_path:
            x = block(x)
            dilated.append(x)
        x = torch.stack(dilated, dim=-1).sum(dim=-1)  # sum over list

        # up-sampling and double convolutions
        for d in range(self.depth):
            x = self.up_path[d](x)
            x = self.decoder_path[d](x, skip[-(d+1)])

        return self.last_conv(x)

In [55]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging
import functools

import torch._utils
import torch.nn.functional as F

BatchNorm2d = functools.partial(nn.BatchNorm2d)
# BatchNorm2d = nn.SynchBatchNorm(InPlaceABNSync, activation='none')

BN_MOMENTUM = 0.01
logger = logging.getLogger(__name__)


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=False)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3 = BatchNorm2d(planes * self.expansion,
                               momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=False)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out


class HighResolutionModule(nn.Module):
    def __init__(self, num_branches, blocks, num_blocks, num_inchannels,
                 num_channels, fuse_method, multi_scale_output=True):
        super(HighResolutionModule, self).__init__()
        self._check_branches(
            num_branches, blocks, num_blocks, num_inchannels, num_channels)

        self.num_inchannels = num_inchannels
        self.fuse_method = fuse_method
        self.num_branches = num_branches

        self.multi_scale_output = multi_scale_output

        self.branches = self._make_branches(
            num_branches, blocks, num_blocks, num_channels)
        self.fuse_layers = self._make_fuse_layers()
        self.relu = nn.ReLU(inplace=False)

    def _check_branches(self, num_branches, blocks, num_blocks,
                        num_inchannels, num_channels):
        if num_branches != len(num_blocks):
            error_msg = 'NUM_BRANCHES({}) <> NUM_BLOCKS({})'.format(
                num_branches, len(num_blocks))
            logger.error(error_msg)
            raise ValueError(error_msg)

        if num_branches != len(num_channels):
            error_msg = 'NUM_BRANCHES({}) <> NUM_CHANNELS({})'.format(
                num_branches, len(num_channels))
            logger.error(error_msg)
            raise ValueError(error_msg)

        if num_branches != len(num_inchannels):
            error_msg = 'NUM_BRANCHES({}) <> NUM_INCHANNELS({})'.format(
                num_branches, len(num_inchannels))
            logger.error(error_msg)
            raise ValueError(error_msg)

    def _make_one_branch(self, branch_index, block, num_blocks, num_channels,
                         stride=1):
        downsample = None
        if stride != 1 or \
                self.num_inchannels[branch_index] != num_channels[branch_index] * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.num_inchannels[branch_index],
                          num_channels[branch_index] * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(num_channels[branch_index] * block.expansion,
                            momentum=BN_MOMENTUM),
            )

        layers = []
        layers.append(block(self.num_inchannels[branch_index],
                            num_channels[branch_index], stride, downsample))
        self.num_inchannels[branch_index] = \
            num_channels[branch_index] * block.expansion
        for i in range(1, num_blocks[branch_index]):
            layers.append(block(self.num_inchannels[branch_index],
                                num_channels[branch_index]))

        return nn.Sequential(*layers)

    def _make_branches(self, num_branches, block, num_blocks, num_channels):
        branches = []

        for i in range(num_branches):
            branches.append(
                self._make_one_branch(i, block, num_blocks, num_channels))

        return nn.ModuleList(branches)

    def _make_fuse_layers(self):
        if self.num_branches == 1:
            return None

        num_branches = self.num_branches
        num_inchannels = self.num_inchannels
        fuse_layers = []
        for i in range(num_branches if self.multi_scale_output else 1):
            fuse_layer = []
            for j in range(num_branches):
                if j > i:
                    fuse_layer.append(nn.Sequential(
                        nn.Conv2d(num_inchannels[j],
                                  num_inchannels[i],
                                  1,
                                  1,
                                  0,
                                  bias=False),
                        BatchNorm2d(num_inchannels[i], momentum=BN_MOMENTUM)))
                elif j == i:
                    fuse_layer.append(None)
                else:
                    conv3x3s = []
                    for k in range(i - j):
                        if k == i - j - 1:
                            num_outchannels_conv3x3 = num_inchannels[i]
                            conv3x3s.append(nn.Sequential(
                                nn.Conv2d(num_inchannels[j],
                                          num_outchannels_conv3x3,
                                          3, 2, 1, bias=False),
                                BatchNorm2d(num_outchannels_conv3x3,
                                            momentum=BN_MOMENTUM)))
                        else:
                            num_outchannels_conv3x3 = num_inchannels[j]
                            conv3x3s.append(nn.Sequential(
                                nn.Conv2d(num_inchannels[j],
                                          num_outchannels_conv3x3,
                                          3, 2, 1, bias=False),
                                BatchNorm2d(num_outchannels_conv3x3,
                                            momentum=BN_MOMENTUM),
                                nn.ReLU(inplace=False)))
                    fuse_layer.append(nn.Sequential(*conv3x3s))
            fuse_layers.append(nn.ModuleList(fuse_layer))

        return nn.ModuleList(fuse_layers)

    def get_num_inchannels(self):
        return self.num_inchannels

    def forward(self, x):
        if self.num_branches == 1:
            return [self.branches[0](x[0])]

        for i in range(self.num_branches):
            x[i] = self.branches[i](x[i])

        x_fuse = []
        for i in range(len(self.fuse_layers)):
            y = x[0] if i == 0 else self.fuse_layers[i][0](x[0])
            for j in range(1, self.num_branches):
                if i == j:
                    y = y + x[j]
                elif j > i:
                    width_output = x[i].shape[-1]
                    height_output = x[i].shape[-2]
                    y = y + F.interpolate(
                        self.fuse_layers[i][j](x[j]),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=False)
                else:
                    y = y + self.fuse_layers[i][j](x[j])
            x_fuse.append(self.relu(y))

        return x_fuse


blocks_dict = {
    'BASIC': BasicBlock,
    'BOTTLENECK': Bottleneck
}


class HighResolutionNet(nn.Module):

    def __init__(self, config, **kwargs):
        extra = config.MODEL.EXTRA
        super(HighResolutionNet, self).__init__()

        # stem net
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1,
                               bias=False)
        self.bn1 = BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1,
                               bias=False)
        self.bn2 = BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=False)

        self.stage1_cfg = extra['STAGE1']
        num_channels = self.stage1_cfg['NUM_CHANNELS'][0]
        block = blocks_dict[self.stage1_cfg['BLOCK']]
        num_blocks = self.stage1_cfg['NUM_BLOCKS'][0]
        self.layer1 = self._make_layer(block, 64, num_channels, num_blocks)
        stage1_out_channel = block.expansion * num_channels

        self.stage2_cfg = extra['STAGE2']
        num_channels = self.stage2_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage2_cfg['BLOCK']]
        num_channels = [
            num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition1 = self._make_transition_layer(
            [stage1_out_channel], num_channels)
        self.stage2, pre_stage_channels = self._make_stage(
            self.stage2_cfg, num_channels)

        self.stage3_cfg = extra['STAGE3']
        num_channels = self.stage3_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage3_cfg['BLOCK']]
        num_channels = [
            num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition2 = self._make_transition_layer(
            pre_stage_channels, num_channels)
        self.stage3, pre_stage_channels = self._make_stage(
            self.stage3_cfg, num_channels)

        self.stage4_cfg = extra['STAGE4']
        num_channels = self.stage4_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage4_cfg['BLOCK']]
        num_channels = [
            num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition3 = self._make_transition_layer(
            pre_stage_channels, num_channels)
        self.stage4, pre_stage_channels = self._make_stage(
            self.stage4_cfg, num_channels, multi_scale_output=True)

        last_inp_channels = np.int(np.sum(pre_stage_channels))

        self.last_layer = nn.Sequential(
            nn.Conv2d(
                in_channels=last_inp_channels,
                out_channels=last_inp_channels,
                kernel_size=1,
                stride=1,
                padding=0),
            BatchNorm2d(last_inp_channels, momentum=BN_MOMENTUM),
            nn.ReLU(inplace=False),
            nn.Conv2d(
                in_channels=last_inp_channels,
                out_channels=config.DATASET.NUM_CLASSES,
                kernel_size=extra.FINAL_CONV_KERNEL,
                stride=1,
                padding=1 if extra.FINAL_CONV_KERNEL == 3 else 0)
        )

    def _make_transition_layer(
            self, num_channels_pre_layer, num_channels_cur_layer):
        num_branches_cur = len(num_channels_cur_layer)
        num_branches_pre = len(num_channels_pre_layer)

        transition_layers = []
        for i in range(num_branches_cur):
            if i < num_branches_pre:
                if num_channels_cur_layer[i] != num_channels_pre_layer[i]:
                    transition_layers.append(nn.Sequential(
                        nn.Conv2d(num_channels_pre_layer[i],
                                  num_channels_cur_layer[i],
                                  3,
                                  1,
                                  1,
                                  bias=False),
                        BatchNorm2d(
                            num_channels_cur_layer[i], momentum=BN_MOMENTUM),
                        nn.ReLU(inplace=False)))
                else:
                    transition_layers.append(None)
            else:
                conv3x3s = []
                for j in range(i + 1 - num_branches_pre):
                    inchannels = num_channels_pre_layer[-1]
                    outchannels = num_channels_cur_layer[i] \
                        if j == i - num_branches_pre else inchannels
                    conv3x3s.append(nn.Sequential(
                        nn.Conv2d(
                            inchannels, outchannels, 3, 2, 1, bias=False),
                        BatchNorm2d(outchannels, momentum=BN_MOMENTUM),
                        nn.ReLU(inplace=False)))
                transition_layers.append(nn.Sequential(*conv3x3s))

        return nn.ModuleList(transition_layers)

    def _make_layer(self, block, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(planes * block.expansion, momentum=BN_MOMENTUM),
            )

        layers = []
        layers.append(block(inplanes, planes, stride, downsample))
        inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(inplanes, planes))

        return nn.Sequential(*layers)

    def _make_stage(self, layer_config, num_inchannels,
                    multi_scale_output=True):
        num_modules = layer_config['NUM_MODULES']
        num_branches = layer_config['NUM_BRANCHES']
        num_blocks = layer_config['NUM_BLOCKS']
        num_channels = layer_config['NUM_CHANNELS']
        block = blocks_dict[layer_config['BLOCK']]
        fuse_method = layer_config['FUSE_METHOD']

        modules = []
        for i in range(num_modules):
            # multi_scale_output is only used last module
            if not multi_scale_output and i == num_modules - 1:
                reset_multi_scale_output = False
            else:
                reset_multi_scale_output = True
            modules.append(
                HighResolutionModule(num_branches,
                                     block,
                                     num_blocks,
                                     num_inchannels,
                                     num_channels,
                                     fuse_method,
                                     reset_multi_scale_output)
            )
            num_inchannels = modules[-1].get_num_inchannels()

        return nn.Sequential(*modules), num_inchannels

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.layer1(x)

        x_list = []
        for i in range(self.stage2_cfg['NUM_BRANCHES']):
            if self.transition1[i] is not None:
                x_list.append(self.transition1[i](x))
            else:
                x_list.append(x)
        y_list = self.stage2(x_list)

        x_list = []
        for i in range(self.stage3_cfg['NUM_BRANCHES']):
            if self.transition2[i] is not None:
                if i < self.stage2_cfg['NUM_BRANCHES']:
                    x_list.append(self.transition2[i](y_list[i]))
                else:
                    x_list.append(self.transition2[i](y_list[-1]))
            else:
                x_list.append(y_list[i])
        y_list = self.stage3(x_list)

        x_list = []
        for i in range(self.stage4_cfg['NUM_BRANCHES']):
            if self.transition3[i] is not None:
                if i < self.stage3_cfg['NUM_BRANCHES']:
                    x_list.append(self.transition3[i](y_list[i]))
                else:
                    x_list.append(self.transition3[i](y_list[-1]))
            else:
                x_list.append(y_list[i])
        x = self.stage4(x_list)

        # Upsampling
        x0_h, x0_w = x[0].size(2), x[0].size(3)
        x1 = F.interpolate(x[1], size=(x0_h, x0_w), mode='bilinear', align_corners=False)
        x2 = F.interpolate(x[2], size=(x0_h, x0_w), mode='bilinear', align_corners=False)
        x3 = F.interpolate(x[3], size=(x0_h, x0_w), mode='bilinear', align_corners=False)

        x = torch.cat([x[0], x1, x2, x3], 1)

        x = self.last_layer(x)

        return x

    def init_weights(self, pretrained='', ):
        logger.info('=> init weights from normal distribution')
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.001)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        if os.path.isfile(pretrained):
            pretrained_dict = torch.load(pretrained)
            logger.info('=> loading pretrained model {}'.format(pretrained))
            model_dict = self.state_dict()
            pretrained_dict = {k: v for k, v in pretrained_dict.items()
                               if k in model_dict.keys()}
            for k, _ in pretrained_dict.items():
                logger.info(
                    '=> loading {} pretrained model {}'.format(k, pretrained))
            model_dict.update(pretrained_dict)
            self.load_state_dict(model_dict)


def get_seg_model(cfg, **kwargs):
    model = HighResolutionNet(cfg, **kwargs)
    model.init_weights(cfg.MODEL.PRETRAINED)

    return model

In [56]:
from yacs.config import CfgNode as CN

_C = CN()

_C.OUTPUT_DIR = ''
_C.LOG_DIR = ''
_C.GPUS = (0,)
_C.WORKERS = 4
_C.PRINT_FREQ = 20
_C.AUTO_RESUME = False
_C.PIN_MEMORY = True
_C.RANK = 0

# Cudnn related params
_C.CUDNN = CN()
_C.CUDNN.BENCHMARK = True
_C.CUDNN.DETERMINISTIC = False
_C.CUDNN.ENABLED = True

# common params for NETWORK
_C.MODEL = CN()
_C.MODEL.NAME = 'seg_hrnet'
_C.MODEL.PRETRAINED = ''
_C.MODEL.EXTRA = CN(new_allowed=True)

_C.LOSS = CN()
_C.LOSS.USE_OHEM = False
_C.LOSS.OHEMTHRES = 0.9
_C.LOSS.OHEMKEEP = 100000
_C.LOSS.CLASS_BALANCE = True

# DATASET related params
_C.DATASET = CN()
_C.DATASET.ROOT = ''
_C.DATASET.DATASET = 'cityscapes'
_C.DATASET.NUM_CLASSES = 19
_C.DATASET.TRAIN_SET = 'list/cityscapes/train.lst'
_C.DATASET.EXTRA_TRAIN_SET = ''
_C.DATASET.TEST_SET = 'list/cityscapes/val.lst'

# training
_C.TRAIN = CN()

_C.TRAIN.IMAGE_SIZE = [1024, 512]  # width * height
_C.TRAIN.BASE_SIZE = 2048
_C.TRAIN.DOWNSAMPLERATE = 1
_C.TRAIN.FLIP = True
_C.TRAIN.MULTI_SCALE = True
_C.TRAIN.SCALE_FACTOR = 16

_C.TRAIN.LR_FACTOR = 0.1
_C.TRAIN.LR_STEP = [90, 110]
_C.TRAIN.LR = 0.01
_C.TRAIN.EXTRA_LR = 0.001

_C.TRAIN.OPTIMIZER = 'sgd'
_C.TRAIN.MOMENTUM = 0.9
_C.TRAIN.WD = 0.0001
_C.TRAIN.NESTEROV = False
_C.TRAIN.IGNORE_LABEL = -1

_C.TRAIN.BEGIN_EPOCH = 0
_C.TRAIN.END_EPOCH = 484
_C.TRAIN.EXTRA_EPOCH = 0

_C.TRAIN.RESUME = False

_C.TRAIN.BATCH_SIZE_PER_GPU = 32
_C.TRAIN.SHUFFLE = True
# only using some training samples
_C.TRAIN.NUM_SAMPLES = 0

# testing
_C.TEST = CN()

_C.TEST.IMAGE_SIZE = [2048, 1024]  # width * height
_C.TEST.BASE_SIZE = 2048

_C.TEST.BATCH_SIZE_PER_GPU = 32
# only testing some samples
_C.TEST.NUM_SAMPLES = 0

_C.TEST.MODEL_FILE = ''
_C.TEST.FLIP_TEST = False
_C.TEST.MULTI_SCALE = False
_C.TEST.SCALE_LIST = [1]

# debug
_C.DEBUG = CN()
_C.DEBUG.DEBUG = False
_C.DEBUG.SAVE_BATCH_IMAGES_GT = False
_C.DEBUG.SAVE_BATCH_IMAGES_PRED = False
_C.DEBUG.SAVE_HEATMAPS_GT = False
_C.DEBUG.SAVE_HEATMAPS_PRED = False


def update_config(file):
    cfg = _C.clone()
    cfg.defrost()
    cfg.merge_from_file(file)
    cfg.freeze()
    return cfg

In [57]:
# Model
if model_name == 'deeplabv3':
    model =  torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=True, num_classes=4)
    # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False, progress=True, num_classes=4)
#elif model_name == 'hrnet_w18':
#    hrnet_cfg = update_config(os.path.join(workspace_path, 'hrnet_w18_config.yaml'))
#    model = get_seg_model(hrnet_cfg)
#elif model_name == 'hrnet_w48':
#    hrnet_cfg = update_config(os.path.join(workspace_path, 'hrnet_w48_config.yaml'))
#    model = get_seg_model(hrnet_cfg)
elif model_name == 'dilated_unet':
    model = MyDilatedConvUNet()

model.to(device)

print('number of parameters: ', count_parameters(model))

number of parameters:  58626628


### 최고 성능 모델 로드

In [58]:
save_path=os.path.join(output_path, 'ckpt')

checkpoint_path = os.path.join(save_path,'{}_best.pt'.format(model_name))
checkpoint = torch.load(checkpoint_path)

model.load_state_dict(checkpoint['model'])
model.to(device)

print('model load success')

model load success


### 테스트 데이터셋 정의

In [59]:
test_rgb_path = os.path.join(workspace_path, 'test/rgb')
test_rgb_images = os.listdir(test_rgb_path)
test_rgb_images = [os.path.join(test_rgb_path, x) for x in test_rgb_images]

In [60]:
#empty value
test_label_path = os.path.join(workspace_path, 'test/label')
try:
    test_label_images = os.listdir(test_label_path)
except:
    test_label_images = []
test_label_images = [os.path.join(test_label_path, x) for x in test_label_images]

In [61]:
test_dataset = CloudDataset(test_rgb_images, test_label_images,
                            transforms=val_transforms, is_train=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False,
                                               num_workers=num_workers, pin_memory=True, drop_last=True)

### 테스트 결과 저장

In [ ]:
model.eval()

result_path = os.path.join(output_path, 'results')
os.makedirs(result_path, exist_ok=True)

with torch.no_grad():
    pbar = tqdm(enumerate(test_dataloader), total=len(test_dataloader))
    for i, (imgs, img_path) in pbar:
        imgs = imgs.to(device)
        if model_name == 'deeplabv3':
            preds = model(imgs)['out']
        elif model_name == 'hrnet_w18' or model_name == 'hrnet_w48':
            preds = model(imgs)
            h, w = preds.shape[2], preds.shape[3]
        elif model_name == 'dilated_unet':
            preds = model(imgs)
        
        pred_img = np.zeros((*list(preds.shape[2:]), 3), dtype=np.uint8)
        _, idx = preds.squeeze(0).max(0)
        pos = idx == 0
        pred_img[pos.cpu().numpy()] = [0, 0, 0]
        pos = idx == 1
        pred_img[pos.cpu().numpy()] = [0, 0, 255]
        pos = idx == 2
        pred_img[pos.cpu().numpy()] = [0, 255, 0]
        pos = idx == 3
        pred_img[pos.cpu().numpy()] = [0, 255, 255]
        
        cv2.imwrite(os.path.join(result_path, os.path.basename(img_path[0])), pred_img)


### Run-Length Encoding

In [ ]:
import pandas as pd

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formatted
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
test_label_file_list = os.listdir(result_path)
test_label_path_list = [os.path.join(result_path, x) for x in test_label_file_list]

In [ ]:
rle_list = []
for file_path in test_label_path_list:
    img = cv2.imread(file_path)
    rle = mask2rle(img)
    rle_list.append(rle)

In [ ]:
submission = pd.read_csv(os.path.join(workspace_path, 'sample_submission.csv'))

In [ ]:
submission['Image_Label'] = test_label_file_list
submission['EncodedPixels'] = rle_list

In [ ]:
submission.to_csv('sample_submission.csv', index=False)

In [ ]:
import shutil
shutil.make_archive('label', 'zip', result_path)

'/home/smkim/CV_CloudsSegmentation/label.zip'

In [ ]:
shutil.rmtree(os.path.join(output_path, 'cache'))
shutil.rmtree(result_path)